# Linear Regression

In [ ]:
from PIL import Image
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('bmh')
matplotlib.rcParams['figure.figsize']=(8,5)

簡易的 linear regression 實驗

In [ ]:
# 產生隨機數據
X = np.random.normal(0, 3, size=(50,1))
Y = X @ [3] + np.random.normal(0, size=50)
# 畫出來看看
plt.plot(X, Y, 'o');

In [ ]:
# 用 numpy 的 lstsq
a = np.linalg.lstsq(X, Y)[0]
a

In [ ]:
# 畫出來
plt.plot(X, Y, 'o')
plt.plot(X, X @ a, 'o');

## Q 
如何加上常數項？

Hint: 使用 `np.concatenate`, `np.ones_like`

In [ ]:
%run -i q_lstsq.py

## 用 sklearn

In [ ]:
from sklearn import linear_model

In [ ]:
X = np.random.normal(0, 3, size=(50,1))
Y = X @ [3] + 4 +np.random.normal(0, size=50)

In [ ]:
regr = linear_model.LinearRegression()
regr

In [ ]:
regr.fit(X,Y)
print(regr.coef_, regr.intercept_)

In [ ]:
# 畫出來
plt.plot(X, Y, 'o')
plt.plot(X, regr.predict(X), 'o');

## Q
畫出 `test_X = np.linspace(-10,10, 100)` 的圖形

In [ ]:
%run -i q_linear_test.py

### 使用 sklearn 的 datasets

In [ ]:
from sklearn import datasets

In [ ]:
diabetes = datasets.load_diabetes()
diabetes

In [ ]:
import scipy.stats

In [ ]:
scipy.stats.describe(diabetes.target)

In [ ]:
idx = np.arange(diabetes.data.shape[0])
np.random.shuffle(idx)
X = diabetes.data[idx]
y = diabetes.target[idx]

試試看 linear regression

In [ ]:
train_X = X[:-50, 2:3]
train_y = y[:-50]
test_X = X[-50:, 2:3]
test_y = y[-50:]
regr = linear_model.LinearRegression()
regr.fit(train_X, train_y)
plt.plot(train_X, train_y, 'o');
plt.plot(train_X, regr.predict(train_X), 'o');
np.mean((regr.predict(train_X)-train_y)**2)

In [ ]:
plt.plot(test_X, test_y, 'o');
plt.plot(test_X, regr.predict(test_X), 'o');

### 用所有變數

In [ ]:
train_X = X[:-50]
train_y = y[:-50]
test_X = X[-50:]
test_y = y[-50:]
regr = linear_model.LinearRegression()
regr.fit(train_X, train_y)
np.mean((regr.predict(train_X)-train_y)**2)

In [ ]:
np.mean((regr.predict(test_X)-test_y)**2)

In [ ]:
plt.plot(test_X[:, 2:3], test_y, 'o');
plt.plot(test_X[:, 2:3], regr.predict(test_X), 'o');

In [ ]:
plt.scatter(regr.predict(train_X), train_y, c='g', s=3)
plt.scatter(regr.predict(test_X), test_y, c='b')
plt.plot([0,300],[0,300],'r', linewidth=1);

In [ ]:
groups = np.arange(30,300,60)
predict_y=regr.predict(train_X)
plt.boxplot([train_y[(predict_y>=i-30)&(predict_y< i+30)] for i in groups], labels=groups);
plt.plot(np.arange(1,len(groups)+1), groups,'x');

## Overfitting

https://tjwei.github.io/NeuralNetwork-Jobspace-slides/#/7

### Regularization
$\frac{1}{2  n} \left\Vert y - Xw\right\Vert_2^2 + α \left\Vert w \right\Vert_1$

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:
regr = linear_model.Lasso(alpha=0.001)
regr.fit(train_X, train_y)
np.mean((regr.predict(train_X)-train_y)**2)

In [ ]:
np.mean((regr.predict(test_X)-test_y)**2)

### Cross validation
https://en.wikipedia.org/wiki/Cross-validation_(statistics)

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [ ]:
from sklearn import model_selection
α_space = np.logspace(-4, 0, 50)
scores =[]
for α in α_space:    
    regr.alpha = α
    s = model_selection.cross_val_score(regr, train_X, train_y, cv=3)
    scores.append((s.mean(), s.std()))
scores=np.array(scores).T
plt.semilogx(α_space, scores[0], 'r')
plt.semilogx(α_space, scores[0]+scores[1],'b--')
plt.semilogx(α_space, scores[0]-scores[1],'b--')
plt.fill_between(α_space, scores[0] + scores[1], scores[0] - scores[1], alpha=0.2);

### Model selection: LassoCV
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

$R^2-score$ : https://en.wikipedia.org/wiki/Coefficient_of_determination#Definitions

In [ ]:
regr = linear_model.LassoCV(alphas = α_space, cv=5)
regr.fit(train_X, train_y)
print('α=', regr.alpha_)
print("training score:", regr.score(train_X, train_y))
# compute R^2 score from definition
print(1-np.mean((regr.predict(train_X)-train_y)**2)/np.var(train_y))

In [ ]:
print("validation score:", regr.score(test_X, test_y))

more about model selection: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html#sphx-glr-auto-examples-linear-model-plot-lasso-model-selection-py

### 用 Linear regression 來 classification ?

In [ ]:
X = np.random.normal(1, size=(100,1))
y = (X[:,0]>0).ravel()*2-1
regr = linear_model.LinearRegression().fit(X, y)
test_X=np.linspace(-3,3,10).reshape(-1,1)
plt.plot(X, y, 'x');
plt.plot(test_X, regr.predict(test_X), 'r')
plt.plot([-regr.intercept_/regr.coef_[0]]*2, [-1.5,1.5], 'r--')
regr.intercept_

In [ ]:
regr.intercept_

## MNIST

In [ ]:
import gzip
import pickle
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')
    
train_X, train_y = train_set
test_X, test_y = test_set

In [ ]:
regr.fit(train_X, train_y)
regr.predict(test_X)

In [ ]:
predict_y = np.floor(regr.predict(train_X)+0.5).astype('int').clip(0,9)
np.mean(predict_y == train_y)

In [ ]:
predict_y = np.floor(regr.predict(test_X)+0.5).astype('int').clip(0,9)
np.mean(predict_y == test_y)

準確率約 23% 很低

### One hot encoding

In [ ]:
train_Y = np.zeros(shape=(train_y.shape[0], 10))
train_Y[np.arange(train_y.shape[0]), train_y] = 1

In [ ]:
train_y[0]

In [ ]:
train_Y[0]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()
onehot_encoder.fit(train_y.reshape(-1,1))
onehot_encoder.transform(train_y.reshape(-1,1)).toarray()[0]

In [ ]:
# 訓練模型
regr.fit(train_X, train_Y)

# 用 argmax 得到結果
predict_y = np.argmax(regr.predict(train_X), axis=1)
# 計算正確率
np.mean(predict_y == train_y)

## Q
試試看 test accuracy

In [ ]:
%run -i q_minst_linear_regression.py

## Q
用 PCA 先處理過 試試看